In [3]:
import os
import json
import shutil
from pathlib import Path
import yaml
import cv2  
# ----------- CONFIG -----------

source_root = "/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/generated"
target_root = "/home/chq29/private/3dgs/4dgs/4DGaussians_farmland/data/farmland"
output_img_dir = target_root
output_intri = os.path.join(target_root, "optimized/intri.yml")
output_extri = os.path.join(target_root, "optimized/extri.yml")
transforms_path = "/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/nerfstudio/nerfstudio_matic20230822_soybean/transforms.json"


resize_scale = 1/4
# ----------- LOAD INTRINSICS FROM transforms.json -----------

with open(transforms_path, "r") as f:
    tf_data = json.load(f)

# Assume all cameras share intrinsics
fx = tf_data["fl_x"] * resize_scale
fy = tf_data["fl_y"] * resize_scale
cx = tf_data["cx"] * resize_scale
cy = tf_data["cy"] * resize_scale
w = tf_data["w"] * resize_scale
h = tf_data["h"] * resize_scale

intri_dict = {
    "K": [
        [fx, 0, cx],
        [0, fy, cy],
        [0,  0,  1]
    ],
    "w": w,
    "h": h
}



# ----------- PREP IMAGE STRUCTURE -----------

os.makedirs(output_img_dir, exist_ok=True)
subdirs = sorted([d for d in Path(source_root).iterdir() if d.is_dir()])

print("day number:", len(subdirs))
print("psoe number:", subdirs)

for day_id, view_dir in enumerate(subdirs):
    day_name = f"frame_{day_id:05d}.jpg"


    # 🔧 Access images/ inside each date folder
    image_dir = view_dir / "images"
    if not image_dir.exists():
        print(f"⚠️  Missing images folder: {image_dir}")
        continue

    image_files = sorted([
        f for f in image_dir.iterdir()
        if f.suffix.lower() in ['.jpg', '.jpeg', '.png']
    ])

    num_views = len(image_files)

    for pose_id, img_path in enumerate(image_files):
        target_name = f"cam{pose_id:02d}"

        target_day_dir = os.path.join(output_img_dir, target_name)
        os.makedirs(target_day_dir, exist_ok=True)

        target_path = os.path.join(target_day_dir, day_name)
        # shutil.copy(str(img_path), target_path)


        # ---------- Resize + Save ----------
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue

        new_w = int(img.shape[1] * resize_scale)
        new_h = int(img.shape[0] * resize_scale)
        resized_img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        cv2.imwrite(target_path, resized_img)




# -------- WRITE extri.yml --------
camera_pose_path = "/home/chq29/private/3dgs/4d_farmland/src/render/nerfstudio/camera_path_822.json"  # 修改为你真实的路径
output_extri_yml = output_extri


with open(camera_pose_path, "r") as f:
    cam_data = json.load(f)

camera_list = cam_data["camera_path"]

names_dict = {
    f"names": [f"{i:02d}" for i in range(num_views)]
}


extri_dict = {}


class InlineList(list): pass

def inline_list_representer(dumper, data):
    return dumper.represent_sequence('tag:yaml.org,2002:seq', data, flow_style=True)

yaml.add_representer(InlineList, inline_list_representer)


def opencv_matrix(rows, cols, data):

    if isinstance(data[0], (list, tuple)):
        flat_data = [float(x) for row in data for x in row]
    else:
        flat_data = [float(x) for x in data]

    return {
        'rows': rows,
        'cols': cols,
        'dt': 'd',
        'data': InlineList(flat_data)
    }


for i, cam in enumerate(camera_list):
    print("i", i)
    cam2world = cam["camera_to_world"]
    Rot = [row[:3] for row in cam2world[:3]]  # 3x3
    T = [row[3] for row in cam2world[:3]]   # 3x1

    extri_dict.update({
        # f"R_{i:02d}": opencv_matrix(3, 1, Rot), # 如果你想同时存Rodrigues向量
        f"Rot_{i:02d}": opencv_matrix(3, 3, Rot),
        f"T_{i:02d}": opencv_matrix(3, 1, [[t] for t in T]),
        f"t_{i:02d}": 0.0000000000
        # f"n_{i}": 0.5,
        # f"f_{i}": 20.0,
    })

# -------- SAVE TO YAML --------
class QuotedStringDumper(yaml.SafeDumper):
    def represent_str(self, data):
        return self.represent_scalar("tag:yaml.org,2002:str", data, style='"')

QuotedStringDumper.add_representer(str, QuotedStringDumper.represent_str)


# with open(output_extri_yml, "w") as f:
#     f.write("%YAML:1.0\n---\n")  # 手动添加头部
#     yaml.dump(names_dict, f, sort_keys=False, Dumper=QuotedStringDumper,)
#     yaml.dump(extri_dict, f, sort_keys=False)

# # ----------- WRITE extri.yml -----------


# ----------- WRITE intri.yml -----------


intri_dict = {}

intri_path = "/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/nerfstudio/nerfstudio_matic20230822_soybean/transforms.json"
with open(intri_path, "r") as f:
    intri_data = json.load(f)


W = intri_data["w"]
H = intri_data["h"]
fx = intri_data["fl_x"]
cx = intri_data["cx"]
fy = intri_data["fl_y"]
cy = intri_data["cy"]



for i, cam in enumerate(camera_list):
    intri_dict.update({
        f"K_{i:02d}": opencv_matrix(3, 3, [fx, 0, cx, 0, fy, cy, 0, 0, 1]), # 如果你想同时存Rodrigues向量
        f"H_{i:02d}": H,
        f"W_{i:02d}": W,
        f"D_{i:02d}": opencv_matrix(5, 1, [0, 0, 0, 0, 0]),
    })


# os.makedirs(os.path.dirname(output_intri), exist_ok=True)
# with open(output_intri, 'w') as f:
#     f.write("%YAML:1.0\n---\n")  # 手动添加头部
#     yaml.dump(names_dict, f, sort_keys=False, Dumper=QuotedStringDumper,)
#     yaml.dump(intri_dict, f, sort_keys=False)







day number: 7
psoe number: [PosixPath('/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/generated/nerfstudio_matic20230822_soybean'), PosixPath('/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/generated/nerfstudio_matic20230827_soybean'), PosixPath('/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/generated/nerfstudio_matic20230831_soybean'), PosixPath('/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/generated/nerfstudio_matic20230906_soybean'), PosixPath('/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/generated/nerfstudio_matic20230911_soybean'), PosixPath('/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/generated/nerfstudio_matic20230917_soybean'), PosixPath('/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/generated/nerfstudio_matic20230922_soybean')]
i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
i 12
i 13
i 14
i 15
i 16
i 17
i 18
i 19
i 20
i 21
i 22
i 23
i 24
i 25
i 26
i 27
i 28
i 29
i 30
i 31
i 32
i 33
i 34
i 35
i 36
i 37
i 38
i 39
i 40
i 41
i 

In [ ]:
##########################NPZ, point cloud
import open3d as o3d
import numpy as np

ply_path = "/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/nerfstudio/nerfstudio_matic20230817_soybean/sparse_pc.ply"
pcd = o3d.io.read_point_cloud(ply_path)
points = np.asarray(pcd.points)

##########################NPZ, point cloud
import open3d as o3d
import numpy as np
def convert_ply_to_npz(ply_path, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    pcd = o3d.io.read_point_cloud(ply_path)
    pts = np.asarray(pcd.points)  # shape (N, 3)
    if pts.shape[1] != 3:
        raise ValueError("point cloud must be Nx3")

    for i in range(7):  # 假设有7帧
        np.savez(os.path.join(out_dir, f"{i:06d}.npz"), points=pts)

convert_ply_to_npz(
    "/home/chq29/private/3dgs/4d_farmland/Dataset/soybean/nerfstudio/nerfstudio_matic20230817_soybean/sparse_pc.ply",
    "/home/chq29/private/3dgs/4dgs/4K4D/data/farmland/surfs"
)